In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVR, SVC

from sklearn.metrics import mean_squared_error, f1_score
from sklearn.metrics import mean_squared_error

In [2]:
# Генерируем уникальный seed
my_code = "Ананьин"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [4]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [9]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [10]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,Species
0,0.105459,0.287726,0.287313,0.298951,0.328553,0.436517,Roach
1,0.430414,0.470825,0.488806,0.536713,0.906803,0.733621,Bream
2,0.001886,0.042254,0.041045,0.041958,0.029187,0.015394,Smelt
3,0.160226,0.287726,0.302239,0.312937,0.407885,0.488684,Whitefish
4,0.020264,0.072435,0.080224,0.094406,0.148315,0.186372,Roach
...,...,...,...,...,...,...,...
90,0.03852,0.128773,0.141791,0.134615,0.175344,0.289225,Perch
91,0.202824,0.406439,0.414179,0.463287,0.747126,0.614703,Bream
92,0.513175,0.555332,0.563433,0.550699,0.625453,0.925262,Perch
93,0.172397,0.295775,0.302239,0.321678,0.438324,0.52674,Roach


In [11]:
# Задание №1 - анализ метода опорных векторов в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [12]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Length2', 'Weight', 'Height']
Width


In [13]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [14]:
y_train


0     0.436517
1     0.733621
2     0.015394
3     0.488684
4     0.186372
        ...   
90    0.289225
91    0.614703
92    0.925262
93     0.52674
94     0.77829
Name: Width, Length: 95, dtype: object

In [15]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
r_model1 = SVR( kernel='linear')
r_model2 = SVR(kernel='poly')
r_model3 = SVR()
r_model4 = SVR(kernel='sigmoid')




In [16]:
r_model1.fit(x_train,y_train)

a=r_model1.predict(x_test)
mse1 = mean_squared_error(y_test, a)
mse1

0.011392474406112962

In [17]:

r_model2.fit(x_train,y_train)
a=r_model2.predict(x_test)
mse2 = mean_squared_error(y_test, a)
mse2

0.025403646877123334

In [18]:
r_model3.fit(x_train,y_train)
a=r_model3.predict(x_test)
mse3 = mean_squared_error(y_test, a)
mse3

0.007220936400430452

In [19]:
r_model4.fit(x_train,y_train)
a=r_model4.predict(x_test)
mse4 = mean_squared_error(y_test, a)
mse4

22.994548826800575

In [20]:
# Задание №2 - анализ метода опорных векторов в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [21]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Width', 'Length3']
Species


In [22]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [23]:
y_train

0         Roach
1         Bream
2         Smelt
3     Whitefish
4         Roach
        ...    
90        Perch
91        Bream
92        Perch
93        Roach
94         Pike
Name: Species, Length: 95, dtype: object

In [24]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
c_model1 = SVC(kernel= 'linear',degree=1)
c_model2 = SVC(kernel='poly',degree=0.5)
c_model3 = SVC(degree=0)
c_model4 = SVC(kernel='sigmoid',degree=1)


In [25]:
c_model1.fit(x_train, y_train)
print(c_model1.predict(x_test))

['Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch']


In [26]:
c_model2.fit(x_train, y_train)
print(c_model2.predict(x_test))

['Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch']


In [27]:
c_model3.fit(x_train, y_train)
print(c_model3.predict(x_test))

['Perch' 'Perch' 'Bream' 'Perch' 'Perch' 'Bream' 'Bream' 'Bream' 'Pike'
 'Bream' 'Perch' 'Perch' 'Pike' 'Perch' 'Pike' 'Perch' 'Smelt' 'Bream'
 'Perch' 'Perch' 'Smelt' 'Smelt' 'Bream' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Smelt' 'Bream']


In [28]:
c_model4.fit(x_train, y_train)
print(c_model4.predict(x_test))

['Perch' 'Perch' 'Perch' 'Bream' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Bream' 'Bream' 'Perch' 'Bream' 'Perch' 'Perch' 'Bream' 'Perch'
 'Perch' 'Bream' 'Smelt' 'Bream' 'Perch' 'Bream' 'Perch' 'Perch' 'Perch'
 'Bream' 'Bream' 'Perch' 'Bream' 'Perch']
